In [1]:
import pandas as pd
import requests
import numpy as np
import time

In [2]:
#importing libraries
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import regex as re
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from bs4 import BeautifulSoup 
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#nltk.download('stopwords')
import warnings
warnings.filterwarnings('ignore')
import pickle

### Renewable test

In [3]:
base_url = 'https://api.pushshift.io/reddit/comment/search'
before_timestamp = round(time.time())
all_dfs1 = pd.DataFrame()
for _ in range(1):
    print(before_timestamp)
    params = {
    "subreddit" : 'RenewableEnergy',
    "size" : 500,
    'after': None,
    'before': before_timestamp
        }
    res = requests.get(base_url,params)
    if res.status_code == 200:
        data = res.json()['data']
        oldest_submission_timestamp = data[-1]['created_utc']
        before_timestamp = oldest_submission_timestamp
        df = pd.DataFrame(data)[["body","subreddit","score"]]
        all_dfs1=all_dfs1.append(pd.DataFrame(df), ignore_index = True)
        
    else:
        print(f'ERROR CODE FOR RES: {res.status_code}')
        break

1587768070


In [4]:
base_url = 'https://api.pushshift.io/reddit/submission/search'
before_timestamp = round(time.time())
all_dfs2 = pd.DataFrame()
for _ in range(1):
    print(before_timestamp)
    params = {
    "subreddit" : 'RenewableEnergy',
    "size" : 500,
    'after': None,
    'before': before_timestamp
        }
    res = requests.get(base_url,params)
    if res.status_code == 200:
        data = res.json()['data']
        oldest_submission_timestamp = data[-1]['created_utc']
        before_timestamp = oldest_submission_timestamp
        df = pd.DataFrame(data)[["title","subreddit","num_comments","score"]]
        all_dfs2=all_dfs2.append(pd.DataFrame(df), ignore_index = True)
    else:
        print(f'ERROR CODE FOR RES: {res.status_code}')
        break

1587768074


In [5]:
all_dfs1.rename(columns={"body":"text"},inplace=True)

In [6]:
all_dfs2.rename(columns={"title":"text"},inplace=True)

In [7]:
df_renewable=all_dfs1.append(all_dfs2,ignore_index=True)

### Energy test

In [8]:
base_url = 'https://api.pushshift.io/reddit/comment/search'
before_timestamp = round(time.time())
all_dfs1 = pd.DataFrame()
for _ in range(1):
    print(before_timestamp)
    params = {
    "subreddit" : 'energy',
    "size" : 500,
    'after': None,
    'before': before_timestamp
        }
    res = requests.get(base_url,params)
    if res.status_code == 200:
        data = res.json()['data']
        oldest_submission_timestamp = data[-1]['created_utc']
        before_timestamp = oldest_submission_timestamp
        df = pd.DataFrame(data)[["body","subreddit","score"]]
        all_dfs1=all_dfs1.append(pd.DataFrame(df), ignore_index = True)
        
    else:
        print(f'ERROR CODE FOR RES: {res.status_code}')
        break

1587768081


In [9]:
base_url = 'https://api.pushshift.io/reddit/submission/search'
before_timestamp = round(time.time())
all_dfs2 = pd.DataFrame()
for _ in range(1):
    print(before_timestamp)
    params = {
    "subreddit" : 'energy',
    "size" : 500,
    'after': None,
    'before': before_timestamp
        }
    res = requests.get(base_url,params)
    if res.status_code == 200:
        data = res.json()['data']
        oldest_submission_timestamp = data[-1]['created_utc']
        before_timestamp = oldest_submission_timestamp
        df = pd.DataFrame(data)[["title","subreddit","num_comments","score"]]
        all_dfs2=all_dfs2.append(pd.DataFrame(df), ignore_index = True)
    else:
        print(f'ERROR CODE FOR RES: {res.status_code}')
        break

1587768085


In [10]:
all_dfs1.rename(columns={"body":"text"},inplace=True)

In [11]:
all_dfs2.rename(columns={"title":"text"},inplace=True)

In [12]:
df_energy=all_dfs1.append(all_dfs2,ignore_index=True)

In [13]:
df=df_energy.append(df_renewable,ignore_index=True)

In [14]:
df=df.fillna(0)

In [15]:
df

,text,subreddit,score,num_comments
0,How you coming up with that?,energy,3,0.0
1,It’s basic thermodynamics you can’t use them f...,energy,3,0.0
2,&gt;Batteries in turn make more renewables pos...,energy,2,0.0
3,The end game has to be batteries at the switch...,energy,1,0.0
4,What does a decarbonized world have to do with...,energy,1,0.0
...,...,...,...,...
1995,Hydrogen trials test carbon-free future for UK...,RenewableEnergy,1,0.0
1996,The Biggest Municipal Solar Farm in the US Is ...,RenewableEnergy,1,2.0
1997,"Invest in renewable energy now, says activist ...",RenewableEnergy,1,0.0
1998,Cry me a river petrol industry. Wah wah wah 😩 ...,RenewableEnergy,1,5.0


## Cleaning and feature engineering

In [16]:
# instantiate the sentiment analyzer
sia = SentimentIntensityAnalyzer()

In [17]:
# Write a function to get the compound sentiment scores for a post
def get_compound_sentiment(post):
    return sia.polarity_scores(post)['compound']

In [18]:
df['sentiment'] = df['text'].apply(get_compound_sentiment)

In [19]:
def text_to_words(raw_review):
    # 1. Remove HTML.
    review_text = BeautifulSoup(raw_review).get_text()
    
    # 2. Remove non-letters.
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    
    # 3. Convert to lower case, split into individual words.
    words = letters_only.lower().split()
    
    # 4. In Python, searching a set is much faster than searching
    # a list, so convert the stopwords to a set.
    stops = set(stopwords.words('english'))#,'energy','renewable'))
    
    # 5. Remove stopwords.
    meaningful_words = [w for w in words if not w in stops]
    
    #6.lemmatizing
    lemmatizer = WordNetLemmatizer()
    lemmmatized_words=[lemmatizer.lemmatize(i) for i in meaningful_words]
    
    # 7. Join the words back into one string separated by space, 
    # and return the result.
    return(" ".join(lemmmatized_words))

In [20]:
# Initialize an empty list to hold the clean text.
clean_text = []
total_reviews = df.shape[0]
for train_review in df['text']:
    
    # Convert review to words, then append to clean_train_reviews.
    clean_text.append(text_to_words(train_review))

In [21]:
df["cleaned_text"]=clean_text

In [22]:
df['cleaned_text'] = df['cleaned_text'].map(lambda x: re.sub('renewable', '', x))

In [23]:
df['cleaned_text'] = df['cleaned_text'].map(lambda x: re.sub('energy', '', x))

In [24]:
df['cleaned_text'] = df['cleaned_text'].map(lambda x: re.sub('renew', '', x))

In [25]:
dirty_word=['http', 'np' ,'reddit','en','wikipedia','com','org','wiki','www','youtube','watch','message']

In [26]:
re_banned_words = re.compile(r"\b(" + "|".join(dirty_word) + ")\\W", re.I)

In [27]:
def RemoveBannedWords(x):
    global re_banned_words
    return re_banned_words.sub("", x)


In [28]:
df['cleaned_text'] = df['cleaned_text'].apply(RemoveBannedWords)

In [29]:
# Let's come up with a list of positive and negative words we might observe.

energy_words = ["oil","gas","hydrogen","plant","fuel","nuclear"]
renewable_energy_words = ["green","wind","geothermal","tidal","grid","storage","solar","biomass","battery"]

In [30]:
def simple_class(text):
     # Instantiate tokenizer.
    tokenizer = RegexpTokenizer(r'\w+')
    
    # Tokenize text.
    tokens = tokenizer.tokenize(text.lower())
    # Count "energy" words.
    energy_count = sum([1 for i in tokens if i in energy_words])
    # Count "renewable energy" words
    renewable_energy_count = sum([1 for i in tokens if i in renewable_energy_words])
    if renewable_energy_count>energy_count:
        return "1"
    else:
        return "0"

In [31]:
df['simpe_class']=df["cleaned_text"].apply(simple_class)

In [32]:
df["simpe_class"] = pd.to_numeric(df["simpe_class"])

In [33]:
df=df[df["cleaned_text"]!=""]

In [34]:
df

,text,subreddit,score,num_comments,sentiment,cleaned_text,simpe_class
0,How you coming up with that?,energy,3,0.0,0.0000,coming,0
1,It’s basic thermodynamics you can’t use them f...,energy,3,0.0,0.0000,basic thermodynamics use peaking,0
2,&gt;Batteries in turn make more renewables pos...,energy,2,0.0,-0.1796,battery turn make s possible stabilize grid lo...,1
3,The end game has to be batteries at the switch...,energy,1,0.0,0.2960,end game battery switching yard neighborhood s...,1
4,What does a decarbonized world have to do with...,energy,1,0.0,0.0000,decarbonized world reality short medium term,0
...,...,...,...,...,...,...,...
1995,Hydrogen trials test carbon-free future for UK...,RenewableEnergy,1,0.0,0.0000,hydrogen trial test carbon free future uk gas ...,0
1996,The Biggest Municipal Solar Farm in the US Is ...,RenewableEnergy,1,2.0,0.0000,biggest municipal solar farm u coming cincinnati,1
1997,"Invest in renewable energy now, says activist ...",RenewableEnergy,1,0.0,-0.4939,invest say activist whose family experienced...,0
1998,Cry me a river petrol industry. Wah wah wah 😩 ...,RenewableEnergy,1,5.0,-0.7579,cry river petrol industry wah wah wah loser,0


In [35]:
df['subreddit'] = [1 if i == 'RenewableEnergy' else 0 for i in df['subreddit']]

In [36]:
true_class=df["subreddit"]

In [37]:
#df.drop(columns=["text","subreddit","score","num_comments"],inplace=True)

In [38]:
df

,text,subreddit,score,num_comments,sentiment,cleaned_text,simpe_class
0,How you coming up with that?,0,3,0.0,0.0000,coming,0
1,It’s basic thermodynamics you can’t use them f...,0,3,0.0,0.0000,basic thermodynamics use peaking,0
2,&gt;Batteries in turn make more renewables pos...,0,2,0.0,-0.1796,battery turn make s possible stabilize grid lo...,1
3,The end game has to be batteries at the switch...,0,1,0.0,0.2960,end game battery switching yard neighborhood s...,1
4,What does a decarbonized world have to do with...,0,1,0.0,0.0000,decarbonized world reality short medium term,0
...,...,...,...,...,...,...,...
1995,Hydrogen trials test carbon-free future for UK...,1,1,0.0,0.0000,hydrogen trial test carbon free future uk gas ...,0
1996,The Biggest Municipal Solar Farm in the US Is ...,1,1,2.0,0.0000,biggest municipal solar farm u coming cincinnati,1
1997,"Invest in renewable energy now, says activist ...",1,1,0.0,-0.4939,invest say activist whose family experienced...,0
1998,Cry me a river petrol industry. Wah wah wah 😩 ...,1,1,5.0,-0.7579,cry river petrol industry wah wah wah loser,0


In [39]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [40]:
X = df['cleaned_text']
y = df['subreddit']

In [41]:
# Split the data into the training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    #stratify=y,
                                                    random_state=42)

In [42]:
X_train

241     may true electric bus today capable driving ro...
813     seychelles build world largest salt water floa...
1242    optimistic nuclear future think cost s go lowe...
1561    lockdown solar cell technology smashed three b...
1085    s replacing coal plant solar company bought co...
                              ...                        
1132    first advice continue study e take master regr...
1296    turn nuclear would great stop gap avoid climat...
861                                      peak oil article
1461    fair happening perverse ban onshore wind farm ...
1128    thanks detailed reply definitely focusing elec...
Name: cleaned_text, Length: 1597, dtype: object

In [43]:
# Instantiate a CountVectorizer.
cvec = CountVectorizer()

In [44]:
# Fit the vectorizer on our corpus.
cvec.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [45]:
# Transform the corpus.
X_train = cvec.transform(X_train)

In [46]:
X_train

<1597x5370 sparse matrix of type '<class 'numpy.int64'>'
	with 22735 stored elements in Compressed Sparse Row format>

In [47]:
# Convert X_train into a DataFrame.

X_train_df = pd.DataFrame(X_train.toarray(),
                          columns=cvec.get_feature_names())
X_train_df

,aabc,aaction,aalborg,ab,abandon,abandoned,abb,abide,ability,able,...,ytg,yukon,yup,zero,zerohedge,zh,zk,znsse,zone,zoning
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1592,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1593,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1594,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1595,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
# Transform test
X_test = cvec.transform(X_test)
X_test_df = pd.DataFrame(X_test.toarray(),
                         columns=cvec.get_feature_names())

X_test_df.head()

,aabc,aaction,aalborg,ab,abandon,abandoned,abb,abide,ability,able,...,ytg,yukon,yup,zero,zerohedge,zh,zk,znsse,zone,zoning
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Now apply the model to test data

In [49]:
# # use pickle.load to load in these assets
# with open('../assets/lon_reg_cvec.pkl', 'rb') as f:
#     m1 = pickle.load(f)

In [50]:
# # use pickle.load to load in these assets
# with open('../assets/lon_reg_tvec.pkl', 'rb') as f:
#     m2 = pickle.load(f)

In [51]:
# predicted_class1=m1.predict(df)

In [52]:
# predicted_class2=m2.predict(df)

In [53]:
# def nice_conmat(y_test, preds, classes):

#     conmat = confusion_matrix(y_test, preds)
#     print(f'Accuracy Score: {accuracy_score(y_test, preds)}')
#     return pd.DataFrame(conmat, columns=['Predicted ' +class_ for class_ in classes], \
#                 index=['Actual '+ class_ for class_ in classes])

In [54]:
# nice_conmat(true_class,predicted_class1,, ['energy', 'renewable_energy'])

In [55]:
#nice_conmat(true_class,predicted_class2,, ['energy', 'renewable_energy'])